# Notebook to define ensemble for production, cm level
Version developed for ViEWS monthly updates: Fatalities002
## Including ensemble weighting

This notebook defines the ensemble used for production: selects a set of pre-trained models, retrieves and calibrates them, computes weights, and computes and stores the ensemble model predictions.

Models are stored in model storage and most of them specified in the notebook fat_cm_constituentmodels

The notebook draws on the following files in this repository:

Script file: 
    Ensembling.py
    FetchData.py

Lists of models:
    ModelList_cm_{dev_id}.csv (not yet functional)
    List of pickles at local directory (will rewrite to drop dependence on this)

# Note
### Numbers in the models 11, 12, 13 are log values even if the column for model 12, 13 is ged_sb_dep



In [25]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook

# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
# import views_dataviz
from views_runs import storage, ModelMetadata
from views_runs.storage import store, retrieve, fetch_metadata
from views_forecasts.extensions import *

#sklearn
from sklearn.metrics import mean_squared_error

# Other packages
import pickle as pkl

pd.set_option('display.float_format', lambda x: '%.6f' % x)

In [26]:
# Packages from this repository, Tools folder
import sys
sys.path.append('../')
sys.path.append('../Tools')
sys.path.append('../Intermediates')
import os

from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated
from FetchData import FetchData, RetrieveFromList
from ViewsEstimators import *


In [27]:
# Common parameters:

dev_id = 'Fatalities003'
run_id = 'Fatalities003'
EndOfHistory = 509
RunGeneticAlgo = True
level = 'cm'
get_future = False

username = os.getlogin()

steps = [*range(1, 36+1, 1)] # Which steps to train and predict for

fi_steps = [1,3,6,12,36]
# Specifying partitions

calib_partitioner_dict = {"train":(121,396),"predict":(397,444)}
test_partitioner_dict = {"train":(121,444),"predict":(445,492)}
future_partitioner_dict = {"train":(121,492),"predict":(493,504)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

Mydropbox = f'/Users/{username}/Dropbox (ViEWS)/ViEWS/'
localpath = f'/Users/{username}/Pickles/'
overleafpath = f'/Users/{username}/Dropbox (ViEWS)/Apps/Overleaf/VIEWS documentation {dev_id}/'

print('User:', username)
print('Dropbox path set to',Mydropbox)
print('Overleaf path set to',overleafpath)
print('Local path set to',localpath)

User: root
Dropbox path set to /Users/root/Dropbox (ViEWS)/ViEWS/
Overleaf path set to /Users/root/Dropbox (ViEWS)/Apps/Overleaf/VIEWS documentation Fatalities003/
Local path set to /Users/root/Pickles/


In [28]:
from ModelDefinitions import DefineEnsembleModels

ModelList = DefineEnsembleModels(level)
    
i = 0
for model in ModelList:
    print(i, model['modelname'], model['data_train'])
    i = i + 1

0 fatalities003_nl_baseline_rf baseline003
1 fatalities003_nl_conflicthistory_rf conflict_ln
2 fatalities003_nl_conflicthistory_hurdle_lgb conflict_ln
3 fatalities003_nl_conflicthistory_long_xgb conflictlong_ln
4 fatalities003_nl_vdem_hurdle_xgb vdem_short
5 fatalities003_nl_wdi_rf wdi_short
6 fatalities003_nl_topics_rf topics_003
7 fatalities003_nl_topics_xgb topics_003
8 fatalities003_nl_topics_hurdle_lgb topics_003
9 fatalities003_nl_joint_broad_rf joint_broad
10 fatalities003_nl_joint_broad_hurdle_rf joint_broad
11 fatalities003_joint_narrow_xgb joint_narrow
12 fatalities003_nl_joint_narrow_hurdle_xgb joint_narrow
13 fatalities003_nl_joint_narrow_hurdle_lgb joint_narrow
14 fatalities003_nl_all_pca3_xgb all_features


In [29]:
stepcols = ['ln_ged_sb_dep']
for step in steps:
    stepcols.append('step_pred_' + str(step))
stored_modelname_calib = level + '_' + ModelList[11]['modelname'] + '_calib'
pd.DataFrame.forecasts.read_store(stored_modelname_calib, run=dev_id)[stepcols]

pr_56_cm_fatalities003_joint_narrow_xgb_calib.parquet


ln_ged_sb_dep  step_pred_1  step_pred_2  step_pred_3  \
month_id country_id                                                         
397      1                0.000000     0.008420     0.011582     0.012673   
         2                0.000000     0.008420     0.011476     0.012418   
         3                0.000000     0.008420     0.011476     0.012418   
         4                0.000000     0.008659     0.011582     0.012673   
         5                0.000000     0.008420     0.011476     0.012418   
...                            ...          ...          ...          ...   
444      242              0.000000     0.017873     0.019947     0.019739   
         243              0.000000     0.008511     0.011476     0.012418   
         244              0.000000     0.009407     0.012381     0.012647   
         245              0.000000     1.889669     2.477441     3.376249   
         246              2.708050     2.604345     2.636136     1.911815   

                     step_pred_4  step_pred_5  step_pred_6  step_pred_7  \
month_id country_id                                                       
397      1              0.014179     0.014431     0.014855     0.016646   
         2              0.014019     0.013617     0.014294     0.016476   
         3              0.014019     0.013617     0.014294     0.015214   
         4              0.014421     0.014431     0.017202     0.045097   
         5              0.014019     0.013617     0.014294     0.016476   
...                          ...          ...          ...          ...   
444      242            0.025537     0.019502     0.023194     0.020523   
         243            0.014019     0.013617     0.014294     0.015214   
         244            0.014203     0.013976     0.014618     0.015567   
         245            2.985935     3.317934     2.572747     3.731551   
         246            2.500160     2.376583     2.074247     2.208894   

                     step_pred_8  step_pred_9  ...  step_pred_27  \
month_id country_id                            ...                 
397      1              0.020506     0.017444  ...      0.024846   
         2              0.021644     0.018288  ...      0.022748   
         3              0.015707     0.018288  ...      0.022748   
         4              0.018661     0.022210  ...      0.074820   
         5              0.021644     0.018288  ...      0.022748   
...                          ...          ...  ...           ...   
444      242            0.021112     0.024462  ...      0.133979   
         243            0.015707     0.015991  ...      0.023141   
         244            0.017467     0.018470  ...      0.061876   
         245            4.061934     4.168920  ...      1.934937   
         246            2.213603     1.467498  ...      2.115453   

                     step_pred_28  step_pred_29  step_pred_30  step_pred_31  \
month_id country_id                                                           
397      1               0.026930      0.027107      0.117257      0.025644   
         2               0.021641      0.022437      0.021148      0.020752   
         3               0.021641      0.022437      0.021148      0.020752   
         4               0.036520      0.056549      0.050284      0.061734   
         5               0.021641      0.022437      0.021148      0.020752   
...                           ...           ...           ...           ...   
444      242             0.135839      0.137115      0.150589      0.164453   
         243             0.022411      0.023122      0.021496      0.021082   
         244             0.107264      0.122521      0.117023      0.057673   
         245             2.045012      1.900755      2.903455      3.046722   
         246             2.292283      2.020375      2.330546      1.196295   

                     step_pred_32  step_pred_33  step_pred_34  step_pred_35  \
month_id country_id                                

# Retrieve and calibrate predictions

In [30]:
# Retrieving the predictions for calibration and test partitions
# The ModelList contains the predictions organized by model

ModelList = RetrieveStoredPredictions(ModelList, steps, EndOfHistory, dev_id, level, get_future)

# ModelList = CalibratePredictions(ModelList, EndOfHistory, steps)

0 fatalities003_nl_baseline_rf
pr_56_cm_fatalities003_nl_baseline_rf_calib.parquet
pr_56_cm_fatalities003_nl_baseline_rf_test.parquet
1 fatalities003_nl_conflicthistory_rf
pr_56_cm_fatalities003_nl_conflicthistory_rf_calib.parquet
pr_56_cm_fatalities003_nl_conflicthistory_rf_test.parquet
2 fatalities003_nl_conflicthistory_hurdle_lgb
pr_56_cm_fatalities003_nl_conflicthistory_hurdle_lgb_calib.parquet
pr_56_cm_fatalities003_nl_conflicthistory_hurdle_lgb_test.parquet
3 fatalities003_nl_conflicthistory_long_xgb
pr_56_cm_fatalities003_nl_conflicthistory_long_xgb_calib.parquet
pr_56_cm_fatalities003_nl_conflicthistory_long_xgb_test.parquet
4 fatalities003_nl_vdem_hurdle_xgb
pr_56_cm_fatalities003_nl_vdem_hurdle_xgb_calib.parquet
pr_56_cm_fatalities003_nl_vdem_hurdle_xgb_test.parquet
5 fatalities003_nl_wdi_rf
pr_56_cm_fatalities003_nl_wdi_rf_calib.parquet
pr_56_cm_fatalities003_nl_wdi_rf_test.parquet
6 fatalities003_nl_topics_rf
pr_56_cm_fatalities003_nl_topics_rf_calib.parquet
pr_56_cm_fatali

In [31]:
ModelList[11]['predictions_calib_df'] = ModelList[11]['predictions_calib_df'].applymap(lambda x: np.exp(x) - 1)
ModelList[11]['predictions_calib_df'].rename(columns={'ln_ged_sb_dep':'ged_sb_dep'}, inplace=True)
ModelList[11]['predictions_test_df'] = ModelList[11]['predictions_test_df'].applymap(lambda x: np.exp(x) - 1)
ModelList[11]['predictions_test_df'].rename(columns={'ln_ged_sb_dep':'ged_sb_dep'}, inplace=True)
ModelList[11]['predictions_test_df']

ged_sb_dep  step_pred_1  step_pred_2  step_pred_3  \
month_id country_id                                                      
445      1             0.000000     0.009167     0.009891     0.012589   
         2             0.000000     0.009167     0.009697     0.012589   
         3             0.000000     0.009167     0.009697     0.011348   
         4             0.000000     0.012901     0.020516     0.017851   
         5             0.000000     0.009167     0.009697     0.012589   
...                         ...          ...          ...          ...   
492      242           0.000000     1.366919     1.430469     0.029422   
         243           0.000000     1.099701     0.009697     0.011619   
         244           0.000000     0.009613     0.011301     0.013539   
         245           0.000000     0.755424     2.479162     4.422010   
         246           0.000000     4.553240     2.658582     4.642549   

                     step_pred_4  step_pred_5  step_pred_6  step_pred_7  \
month_id country_id                                                       
445      1              0.013190     0.013403     0.015648     0.015318   
         2              0.013058     0.013226     0.014898     0.015174   
         3              0.013058     0.013226     0.014898     0.015174   
         4              0.025427     0.028214     0.028537     0.023996   
         5              0.013058     0.013226     0.014898     0.015174   
...                          ...          ...          ...          ...   
492      242            0.032619     0.030597     0.042787     0.046214   
         243            0.013058     0.013226     0.015052     0.015174   
         244            0.014310     0.014538     0.015642     0.015674   
         245            7.200095     5.574021     2.070196     0.294606   
         246            7.120520     6.137676     6.905732     3.119195   

                     step_pred_8  step_pred_9  ...  step_pred_27  \
month_id country_id                            ...                 
445      1              0.028433     0.018103  ...      0.024727   
         2              0.028433     0.018103  ...      0.019271   
         3              0.014908     0.014455  ...      0.019271   
         4              0.037756     0.020045  ...      0.030460   
         5              0.028433     0.018103  ...      0.019271   
...                          ...          ...  ...           ...   
492      242            0.057206     0.067597  ...      0.157345   
         243            0.014908     0.014812  ...      0.019453   
         244            0.018947     0.014960  ...      0.039761   
         245            0.310444     0.391680  ...     12.175900   
         246            2.875350     3.554297  ...      7.959874   

                     step_pred_28  step_pred_29  step_pred_30  step_pred_31  \
month_id country_id                                                           
445      1               0.034935      0.030096      0.031895      0.026588   
         2               0.019915      0.021934      0.020112      0.019567   
         3               0.019915      0.021934      0.020112      0.019567   
         4               0.030369      0.035826      0.052080      0.057105   
         5               0.019915      0.021934      0.020112      0.019567   
...                           ...           ...           ...           ...   
492      242             0.160875      0.168210      0.171885      0.185036   
         243             0.020766      0.021934      0.020112      0.019906   
         244             0.042759      0.042088      0.042418      0.042666   
         245            10.848603     36.012710     27.432978     24.526728   
         246            12.850042     14.084732     24.437838     20.904996   

                     step_pred_32  step_pred_33  step_pred_34  step_pred_35  \
month_id country_id                                                           
445      1 

In [32]:
ModelList[12]['predictions_calib_df'] = ModelList[12]['predictions_calib_df'].applymap(lambda x: np.exp(x) - 1)
ModelList[13]['predictions_calib_df'] = ModelList[13]['predictions_calib_df'].applymap(lambda x: np.exp(x) - 1)
ModelList[12]['predictions_test_df'] = ModelList[12]['predictions_test_df'].applymap(lambda x: np.exp(x) - 1)
ModelList[13]['predictions_test_df'] = ModelList[13]['predictions_test_df'].applymap(lambda x: np.exp(x) - 1)


# Evaluate models

In [33]:
prediction_df = 'predictions_calib_df'
# prediction_df = 'predictions_test_df'

In [34]:
def calculate_mse(ModelList, prediction_df):
    for model in ModelList:
        df = model[prediction_df]
        pred_cols = [f'step_pred_{str(i)}' for i in steps]
        df['mse'] = df.apply(lambda row: mean_squared_error([row['ged_sb_dep']] * 36, 
                            [row[col] for col in pred_cols]), axis=1)

def get_model_mse(ModelList, prediction_df):
    model_mse = {'model': [], 'mse': []}
    for model in ModelList:
        name = model['modelname']
        df = model[prediction_df]
        model_mse['model'].append(name)
        model_mse['mse'].append(df['mse'].mean()) 
    df_model_mse = pd.DataFrame(model_mse)
    return df_model_mse

def get_top_10_cases(ModelList, prediction_df):
    top_10_cases = {'model': [], 'month_id': [], 'country_id': [], 'mse': []}
    for model in ModelList:
        name = model['modelname']
        df = model[prediction_df]
        df_sorted_model = df.sort_values(by=['mse'], ascending=False).head(10)
        for _ in range(10):
            top_10_cases['model'].append(name)
        for month in df_sorted_model.index.get_level_values(level=0):
            top_10_cases['month_id'].append(month)
        for country in df_sorted_model.index.get_level_values(level=1):
            top_10_cases['country_id'].append(country)  
        for mse in df_sorted_model['mse']:
            top_10_cases['mse'].append(mse)
    pd_top_10_cases = pd.DataFrame(top_10_cases)
    pd_top_10_cases.set_index('model', inplace=True)
    return pd_top_10_cases


In [35]:
calculate_mse(ModelList, 'predictions_calib_df')
calculate_mse(ModelList, 'predictions_test_df')

## calib_df

In [37]:
get_model_mse(ModelList, 'predictions_calib_df') 

,model,mse
0,fatalities003_nl_baseline_rf,226794.434086
1,fatalities003_nl_conflicthistory_rf,233468.644101
2,fatalities003_nl_conflicthistory_hurdle_lgb,208824.486308
3,fatalities003_nl_conflicthistory_long_xgb,527213.482069
4,fatalities003_nl_vdem_hurdle_xgb,710133.395333
5,fatalities003_nl_wdi_rf,218310.056328
6,fatalities003_nl_topics_rf,270199.737508
7,fatalities003_nl_topics_xgb,478322.044438
8,fatalities003_nl_topics_hurdle_lgb,229854.243255
9,fatalities003_nl_joint_broad_rf,301660.673825


In [38]:
get_model_mse(ModelList, 'predictions_calib_df').sort_values(by=['mse'])

,model,mse
2,fatalities003_nl_conflicthistory_hurdle_lgb,208824.486308
5,fatalities003_nl_wdi_rf,218310.056328
13,fatalities003_nl_joint_narrow_hurdle_lgb,225942.525280
0,fatalities003_nl_baseline_rf,226794.434086
12,fatalities003_nl_joint_narrow_hurdle_xgb,228206.830828
11,fatalities003_joint_narrow_xgb,228245.116150
8,fatalities003_nl_topics_hurdle_lgb,229854.243255
1,fatalities003_nl_conflicthistory_rf,233468.644101
6,fatalities003_nl_topics_rf,270199.737508
9,fatalities003_nl_joint_broad_rf,301660.673825


In [39]:
get_top_10_cases(ModelList, 'predictions_calib_df')

,month_id,country_id,mse
model,,,
fatalities003_nl_baseline_rf,420,220,343347462.743495
fatalities003_nl_baseline_rf,408,220,339572761.048223
fatalities003_nl_baseline_rf,444,220,288867512.324259
fatalities003_nl_baseline_rf,432,220,276694196.499313
fatalities003_nl_baseline_rf,398,220,31257369.266348
...,...,...,...
fatalities003_nl_all_pca3_xgb,422,70,71738516.922842
fatalities003_nl_all_pca3_xgb,434,220,54954921.294722
fatalities003_nl_all_pca3_xgb,433,220,53884778.173239


## test_df

In [40]:
get_model_mse(ModelList, 'predictions_test_df') 

,model,mse
0,fatalities003_nl_baseline_rf,49273.511126
1,fatalities003_nl_conflicthistory_rf,160455.801483
2,fatalities003_nl_conflicthistory_hurdle_lgb,97776.831939
3,fatalities003_nl_conflicthistory_long_xgb,665908.011413
4,fatalities003_nl_vdem_hurdle_xgb,96018.694591
5,fatalities003_nl_wdi_rf,48650.088309
6,fatalities003_nl_topics_rf,164361.410425
7,fatalities003_nl_topics_xgb,256316.802069
8,fatalities003_nl_topics_hurdle_lgb,66437.400058
9,fatalities003_nl_joint_broad_rf,328805.902059


In [41]:
get_model_mse(ModelList, 'predictions_test_df').sort_values(by=['mse'])

,model,mse
13,fatalities003_nl_joint_narrow_hurdle_lgb,32225.830488
12,fatalities003_nl_joint_narrow_hurdle_xgb,34570.185765
11,fatalities003_joint_narrow_xgb,35070.568879
5,fatalities003_nl_wdi_rf,48650.088309
0,fatalities003_nl_baseline_rf,49273.511126
8,fatalities003_nl_topics_hurdle_lgb,66437.400058
4,fatalities003_nl_vdem_hurdle_xgb,96018.694591
2,fatalities003_nl_conflicthistory_hurdle_lgb,97776.831939
10,fatalities003_nl_joint_broad_hurdle_rf,112196.560290
1,fatalities003_nl_conflicthistory_rf,160455.801483


In [43]:
get_top_10_cases(ModelList, 'predictions_test_df')

,month_id,country_id,mse
model,,,
fatalities003_nl_baseline_rf,491,126,35522881.008833
fatalities003_nl_baseline_rf,475,133,11948512.565489
fatalities003_nl_baseline_rf,477,133,11072773.401089
fatalities003_nl_baseline_rf,489,213,9913754.491375
fatalities003_nl_baseline_rf,468,149,9209752.885938
...,...,...,...
fatalities003_nl_all_pca3_xgb,461,220,81984914.902293
fatalities003_nl_all_pca3_xgb,469,220,81275477.184191
fatalities003_nl_all_pca3_xgb,470,220,79672313.412752
